[983. Minimum Cost For Tickets](https://leetcode.com/problems/minimum-cost-for-tickets/description/)

The problem can be solved using dynamic programming.  
We can define a one-dimensional array `dp` of length `n+1`, where `n` is the largest day in the `days` array.  
Each element `dp[i]` of the array represents the `minimum cost to travel up to day i`. 

We can then iterate over the elements of dp and update them according to the minimum of the following three options:
* Buy a one-day ticket on day i, in which case the cost is dp[i-1] + costs[0].  
* Buy a seven-day ticket that is valid starting from day i-6, in which case the cost is dp[max(0,i-7)] + costs[1].  
* Buy a thirty-day ticket that is valid starting from day i-29, in which case the cost is dp[max(0,i-30)] + costs[2].  

After iterating over the entire dp array, the final minimum cost will be stored in the last element `dp[-1]`.  

The idea behind the dynamic programming approach is to build up the solution incrementally,   
starting from smaller subproblems and working our way up to larger subproblems.  

In this case, we start by considering the minimum cost to travel up to day 1,   
which is simply costs[0] if day 1 is in the days array, or 0 if it is not.  

We then consider the minimum cost to travel up to day 2, which can be computed by considering the minimum cost  
of buying a one-day ticket on day 2, a seven-day ticket valid from day 1 to day 7, or a thirty-day ticket valid from day 1 to day 30.  
 
We can then use this information to compute the minimum cost to travel up to day 3, and so on.  

By iteratively building up the minimum cost to travel up to each day, we eventually arrive at the minimum cost  
to travel up to the largest day in the days array, which is our final solution.

In [30]:
class Solution:
    def mincostTickets(self, days, costs):
        
        ticket_duration = [1,7,30]
        r = len(ticket_duration)
        k = max(days) + 1       # start indexing from 1, 0 as initial state
        dp = [0] * k            #  the minimum cost to travel up to day i. [0, 0, 0, 0 ...]

        travel_days = set(days) # use a set for quick iterating
        for i in range(1, k):   # iterate over dp  elements, actually over all range of days

            if i not in travel_days:
                dp[i] = dp[i-1] # copy previous cost

            else:  # choose the minima among 3 variants
                #       clipping index by 0, select cost backwards by ticket duration and add cost for that ticket
                dp[i] = min( [dp[max(0, i - ticket_duration[ j ]) ]               + costs[ j ] for j in range( r ) ] )

        return dp[-1]

In [31]:
days = [1,4,6,     9,10,11,12,13,14,15,     16,17,18,20,21,22,  23,27,28]    # sum  44
costs = [3,13,45]
sol = Solution()
sol.mincostTickets(days, costs)

44

In [16]:
print ('cost'+''.join(str(dp)))
print ('days'+''.join(str(temp)))
print (''.join(str(days)))
# cost[0, 3, 3, 3, 6, 6, 9, 9, 9, 12, 15, 18, 19, 22, 22, 22, 25, 28, 31, 31, 34, 35, 35, 38, 38, 38, 38, 41, 44]
# days[0, 1, 1, 1, 4, 4, 6, 6, 6,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 20, 21, 22, 23, 23, 23, 23, 27, 28]
#     [1,          4,    6,        9, 10, 11, 12, 13, 14, 15, 16, 17, 18,     20, 21, 22, 23,             27, 28]

cost[0, 3, 3, 3, 6, 6, 9, 9, 9, 12, 15, 18, 19, 22, 22, 22, 25, 28, 31, 31, 34, 35, 35, 38, 38, 38, 38, 41, 44]
days[0, 1, 1, 1, 4, 4, 6, 6, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 20, 21, 22, 23, 23, 23, 23, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 2